# TF-IDF Scripts

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk import word_tokenize, pos_tag_sents, pos_tag
import re
from bs4 import BeautifulSoup
import unicodedata
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import os, glob


### Custom Functions and File Imports

In [3]:
def removeStopWords(text):
    pass
    

In [4]:
os.chdir("D:\Programs\Jupyter Notebooks\Thesis RIP Scripts\Annotations\Batch 004") # CHANGE VALUE PER BATCH
files = []
for file in glob.glob("*.csv"):
    print(file)
    files.append(file)
    
print(files)    

aless_enri.csv
faye_posadas.csv
kate_ermitano.csv
['aless_enri.csv', 'faye_posadas.csv', 'kate_ermitano.csv']


In [5]:
coder1 = pd.read_csv(files[0])
coder1.fillna(0)
coder2 = pd.read_csv(files[1])
coder2.fillna(0)
coder3 = pd.read_excel('usanews0full_3.xlsx')
coder3.fillna(0)

,Unnamed: 0,message,author,time_text,Direct Harassment,Hate Speech,Sexual Harassment,Trolling,Others,Toxic
0,9000,👀YUP ! FACTS! PROOF! EVIDENCE! TRUTH! IS THE L...,username@ISRKEWKELW,1900-01-02 02:15:00,0.0,0.0,0.0,0.0,0.0,0.0
1,9001,"Pray USA, 🙏🙏🙏❤️❤️❤️",username@BnnBe eeBn,1900-01-02 02:15:00,0.0,0.0,0.0,0.0,0.0,0.0
2,9002,TRUMP 2020❤️❤️❤️❤️RED WAVE INCOMING,username@iizzilln i,1900-01-02 02:15:00,0.0,0.0,0.0,0.0,0.0,0.0
3,9003,All you people crying about covid dont even kn...,username@JxeoonFio,1900-01-02 02:16:00,0.0,1.0,0.0,0.0,1.0,1.0
4,9004,GERMANY LOVES TRUMP! WE NEED SOMEONE LIKE HIM!,username@rraorroaag,1900-01-02 02:16:00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2995,11995,Coma-la Harris is Evil,username@oR ss oBBs,01:03:36,1.0,0.0,0.0,1.0,0.0,0.0
2996,11996,:awesome: :awesome: :awesome:,username@riiñrJoroo,01:03:36,0.0,0.0,0.0,0.0,0.0,0.0
2997,11997,BIDEN WILL DEFUND THE POLICE AND RADICAL LEFT ...,username@dsenesea t,01:03:36,0.0,0.0,0.0,1.0,0.0,0.0
2998,11998,":thanksdoc::thanksdoc::thanksdoc:THANK YOU, ME...",username@NSaimmmoaN,01:03:36,0.0,0.0,0.0,0.0,0.0,0.0


### Combine Documents Per Label

In [6]:
dHarassment = pd.DataFrame()
temp0 = coder1.loc[coder1['Direct Harassment'] == 1.0]
temp1 = coder2.loc[coder2['Direct Harassment'] == 1.0]
temp2 = coder3.loc[coder3['Direct Harassment'] == 1.0]

dHarassment = pd.concat([temp0, temp1, temp2])
dHarassment.drop(dHarassment.columns.difference(['Unnamed: 0', 'message','Direct Harassment']), 1, inplace=True)

dHarassment.drop_duplicates(subset ="message", keep = False, inplace = True)
dHarassment
#duplicateRowsDF = dHarassment[dHarassment.duplicated()]
#duplicateRowsDF

,Unnamed: 0,message,Direct Harassment
48,9048,Biden is a corpse,1.0
53,9053,BEIJING BIDEN FOR PRISON!,1.0
62,9062,MEDIA MAFIA 4 BIDEN,1.0
81,9081,Trump is the best thing that could happen to g...,1.0
87,9087,Biden sucks,1.0
...,...,...,...
2692,11692,the so called news reporters are not real prof...,1.0
2767,11767,LADY GAGA IS REALLY A MALE,1.0
2843,11843,Where is Hunter? Why isn’t he helping Sleepy?,1.0
2895,11895,SANTA'S COMING TO TOWN YOU FILTHY SONE A BEEATCH,1.0


In [7]:
hSpeech = pd.DataFrame()
temp0 = coder1.loc[coder1['Hate Speech'] == 1.0]
temp1 = coder2.loc[coder2['Hate Speech'] == 1.0]
temp2 = coder3.loc[coder3['Hate Speech'] == 1.0]

hSpeech = pd.concat([temp0, temp1, temp2])
hSpeech.drop(hSpeech.columns.difference(['Unnamed: 0', 'message','Hate Speech']), 1, inplace=True)

hSpeech.drop_duplicates(subset ="message", keep = False, inplace = True)
hSpeech

,Unnamed: 0,message,Hate Speech
33,9033,CHINA BUYDEN,1.0
53,9053,BEIJING BIDEN FOR PRISON!,1.0
127,9127,please usa vote trump anti china,1.0
147,9147,🧱******BIDEN WILL RISE*******🧱 ***************...,1.0
163,9163,people who vote for Biden are simply uneducate...,1.0
...,...,...,...
2685,11685,"DEMS ONLY FAKE ""CARING"" TO BAIT PEOPLE & USE T...",1.0
2780,11780,🧂BIDEN SUPPORTERS ONLY WANT FREE SHIT🧂,1.0
2831,11831,PA will GO TRUMP/GOP. Our Dem GOV is a moron.,1.0
2916,11916,DEMOCRATS HATE AMERICA,1.0


In [8]:
sHarassment = pd.DataFrame()
temp0 = coder1.loc[coder1['Sexual Harassment'] == 1.0]
temp1 = coder2.loc[coder2['Sexual Harassment'] == 1.0]
temp2 = coder3.loc[coder3['Sexual Harassment'] == 1.0]

sHarassment = pd.concat([temp0, temp1, temp2])
sHarassment.drop(sHarassment.columns.difference(['Unnamed: 0', 'message','Sexual Harassment']), 1, inplace=True)

sHarassment.drop_duplicates(subset ="message", keep = False, inplace = True)
sHarassment

,Unnamed: 0,message,Sexual Harassment
24,9024,Lady GaGa is Ugly GaGa another weak minded per...,1.0
126,9126,THE SPIRIT IS STRONG WITH THE GREY HAIR.,1.0
552,9552,HARRIS needs her broom stick,1.0
943,9943,T-REX MOUTH KAMALA,1.0
1086,10086,Keep America Free!!,1.0
1260,10260,GRAB PENCE BY THE P**** !!! 💥💥,1.0
1664,10664,🦶HUNTERS FOOTJOBS MATTER 🍌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌🛌...,1.0
2171,11171,🦶HUNTERS FOOTJOBS MATTER 🍌,1.0


### NLP Text Processing Stuff


### TFIDF Stuff